In [1]:
from math import ceil


def chebyshev_distance(src: tuple[int, int], dest: tuple[int, int]):
    return ceil(max(abs((src[0] - dest[0]) / 2.), abs((src[1] - dest[1]) / 2.)))


dest = (2, 6)

maze = [
    [
        chebyshev_distance((j, i), dest) for j in range(0, 7+1)
    ] for i in range(0, 7+1)
]
maze

[[3, 3, 3, 3, 3, 3, 3, 3],
 [3, 3, 3, 3, 3, 3, 3, 3],
 [2, 2, 2, 2, 2, 2, 2, 3],
 [2, 2, 2, 2, 2, 2, 2, 3],
 [1, 1, 1, 1, 1, 2, 2, 3],
 [1, 1, 1, 1, 1, 2, 2, 3],
 [1, 1, 0, 1, 1, 2, 2, 3],
 [1, 1, 1, 1, 1, 2, 2, 3]]

In [9]:
Coords = tuple[int, int]
Step = tuple[Coords, Coords, int, int, int]


open_path: list[Step] = []
closed_path: list[Step] = []


def make_step(next_coord: Coords, prev_node: Step, h: Coords) -> Step:
    g = prev_node[2] + 1
    h = chebyshev_distance(next_coord, h)
    return (next_coord, prev_node[0], g, h, g + h)


def format_step(step: Step) -> str:
    from_coord = f"[{step[1][0]}, {step[1][1]}]" if step[1] else "NULL"
    return f"([{step[0][0]}, {step[0][1]}], {from_coord}, {step[2]}, {step[3]}, {step[4]})"


def horse_pathfind(src: tuple[int, int], dest: tuple[int, int]) -> list[Step]:
    iter_no = 0
    closed_path.clear()
    open_path.clear()

    src_dist = chebyshev_distance(src, dest)
    src_path: Step = (src, None, 0, src_dist, src_dist)

    open_path.append(src_path)

    while open_path:
        iter_no += 1
        
        # Choose the first cheapest path to process
        next_step = open_path[0]
        for p in open_path:
            if p[4] < next_step[4]:
                next_step = p

        print(f"Iteration {iter_no}")
        print(f"Processing: {format_step(next_step)}")

        next_path_coords = next_step[0]
        found_path = next_path_coords == dest
        if not found_path:
            expand_path(next_step, dest)

        closed_path.append(next_step)
        open_path.remove(next_step)

        print("Open:")
        print(";".join(format_step(p) for p in open_path))
        print("Closed:")
        print(";".join(format_step(p) for p in closed_path))
        print()

        if found_path:
            return closed_path
    

    raise Exception("No path found")


step_options = [(-1, -2), (1, -2), (-2, -1), (2, -1), (-2, 1), (2, 1), (-1, 2), (1, 2)]


def _get_next_paths(src: Step):
    x = src[0][0]
    y = src[0][1]
    next_paths = [
        (x + opt[0], y + opt[1])
        for opt in step_options
        if (0 <= x + opt[0]) and (x + opt[0] <= 7) and (0 <= y + opt[1]) and (y + opt[1] <= 7)
    ]
    return next_paths


def expand_path(src: Step, dest: Coords):
    next_coords = _get_next_paths(src)

    closed_coords = [p[0] for p in closed_path]
    next_coords = [p for p in next_coords if p not in closed_coords]

    for coord in next_coords:
        f = chebyshev_distance(coord, dest)

        existing_paths = [p for p in open_path if p[0] == coord]
        if existing_paths:
            assert len(existing_paths) == 1
            if f < existing_paths[0][3]:
                open_path.remove(existing_paths[0])
                print(f"Coord {coord} is open and is worse. Replacing")
            else:
                print(f"Coord {coord} is already open and is better. Skipping")
                continue

        open_path.append(make_step(coord, src, dest))

            



In [7]:
open_path: list[Step] = []
closed_path: list[Step] = []

src = (2, 6)
dest = (2, 6)
dist = chebyshev_distance(src, dest)
print(dist)

0


In [ ]:

src = (3, 7)
dest = (3, 2)
dist = chebyshev_distance(src, dest)

res = horse_pathfind(src, dest).copy()

print()

res = res[::-1]
[print(r) for r in res]

print()

last_step = res.pop(0)
print(last_step)

res_str = ""

for path in res:
    if last_step[1] == path[0]:
        print(last_step[0])
        res_str += f"[{last_step[0][0]}, {last_step[0][1]}]; "
        last_step = path

print(res_str)

Iteration 1
Processing: ([3, 7], NULL, 0, 3, 3)
Open:
([2, 5], [3, 7], 1, 2, 3);([4, 5], [3, 7], 1, 2, 3);([1, 6], [3, 7], 1, 2, 3);([5, 6], [3, 7], 1, 2, 3)
Closed:
([3, 7], NULL, 0, 3, 3)

Iteration 2
Processing: ([2, 5], [3, 7], 1, 2, 3)
Open:
([4, 5], [3, 7], 1, 2, 3);([1, 6], [3, 7], 1, 2, 3);([5, 6], [3, 7], 1, 2, 3);([1, 3], [2, 5], 2, 1, 3);([3, 3], [2, 5], 2, 1, 3);([0, 4], [2, 5], 2, 2, 4);([4, 4], [2, 5], 2, 1, 3);([0, 6], [2, 5], 2, 2, 4);([4, 6], [2, 5], 2, 2, 4);([1, 7], [2, 5], 2, 3, 5)
Closed:
([3, 7], NULL, 0, 3, 3);([2, 5], [3, 7], 1, 2, 3)

Iteration 3
Processing: ([4, 5], [3, 7], 1, 2, 3)
Coord (3, 3) is already open and is better. Skipping
Open:
([1, 6], [3, 7], 1, 2, 3);([5, 6], [3, 7], 1, 2, 3);([1, 3], [2, 5], 2, 1, 3);([3, 3], [2, 5], 2, 1, 3);([0, 4], [2, 5], 2, 2, 4);([4, 4], [2, 5], 2, 1, 3);([0, 6], [2, 5], 2, 2, 4);([4, 6], [2, 5], 2, 2, 4);([1, 7], [2, 5], 2, 3, 5);([5, 3], [4, 5], 2, 1, 3);([2, 4], [4, 5], 2, 1, 3);([6, 4], [4, 5], 2, 2, 4);([2, 6], [4, 